# Actions and Orbit caluclation with MC sampling for GALAH DR3

## Author: Sven Buder

### History:
181011 SB Created  
190222 SB Included sampling with 5D covariance matrix and fixed galpy coordinate transformation for J2015.5 in ICRS

In [ ]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import sys
import glob
import pickle
import collections
import pandas

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
import astropy.coordinates as coord
import astropy.units as u
import math
from astropy.table import Table, hstack, vstack
from scipy.io.idl import readsav

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
    'font.family'        : 'sans',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]

In [ ]:
debug = False

### Galpy initialization

In [ ]:
import galpy
from galpy.potential import MWPotential2014 as pot
from galpy.actionAngle import actionAngleStaeckel
from galpy.util import bovy_coords
from galpy.orbit import Orbit

#galpy scale units:                                                                                                                                                                                         
_REFR0 = 8.178 #[kpc] --> galpy length unit, reference: https://arxiv.org/abs/1904.05721                                                                                                                                                   
_REFV0 = 229. #[km/s] --> galpy velocity unit, reference: https://arxiv.org/abs/1810.09466

aAS = actionAngleStaeckel(
        pot   = pot,        #potential                                                                                                                                                                      
        delta = 0.45,       #focal length of confocal coordinate system                                                                                                                            
        c     = True        #use C code (for speed)                                                                                                                                                         
        )

print("galpy scale units are _REFR0 = "+str(_REFR0)+" kpc and _REFV0 = "+str(_REFV0)+" km/s.")
print("The Sun has an angular momentum of "+str(_REFR0 * _REFV0)+" kpc km/s in this action framework with MWPotential2014")

#Galactocentric position of the Sun according to Gravity Collaboration / Bland-Hawthorn & Gerhard (2016)
X_gc_sun_kpc = _REFR0 #[kpc]
Z_gc_sun_kpc = 0.025 #[kpc]

print(r"We place Sgr A at (x; y; z) = (R_0; 0; z_0) kpc, where")
print("R_0 = "+str(X_gc_sun_kpc)+" kpc and z_0 = "+str(Z_gc_sun_kpc)+" pc (Bland-Hawthorn & Gerhard, 2016)")

#Velocity of the Sun w.r.t. the Local Standard of Rest (e.g. Schoenrich et al. 2009):
U_LSR_kms = 11.1  # [km/s]
V_LSR_kms = 12.24 # [km/s]
W_LSR_kms = 7.25  # [km/s]

#Galactocentric velocity of the Sun:                                                                                                                                                                        
vX_gc_sun_kms = -U_LSR_kms           # = -U              [km/s]
vY_gc_sun_kms =  V_LSR_kms+_REFV0    # = V+v_circ(R_Sun) [km/s]
vZ_gc_sun_kms =  W_LSR_kms           # = W               [km/s]

print("The Sun's velocity with respect to a co-located particle on a circular orbit is")
print("V_LSR = (U_sun, V_sun, W_sun) = ("+str(vX_gc_sun_kms)+", "+str(vY_gc_sun_kms-_REFV0)+", "+str(vZ_gc_sun_kms)+") km/s (Schoenrich 2012)")

### Input of 6D information in observable dimensions

In [ ]:
# Where is sobject_iraf_53_2MASS_GaiaDR2_WISE_PanSTARRSDR1_BailerJones_K2seis.fits?

#file_directory = '/shared-storage/buder/svn-repos/trunk/GALAH/GALAH_DR3/'
file_directory = '/Users/svenbuder/GALAH_DR3'

In [ ]:
#galah_gaia_input = pyfits.getdata(file_directory+'/input/sobject_iraf_53_2MASS_GaiaDR2_WISE_PanSTARRSDR1_BailerJones_K2seis.fits',1)
galah_gaia_input = pyfits.getdata(file_directory+'/catalogs/GALAH_DR3_main.fits',1)

full_length = len(galah_gaia_input['sobject_id'])
print("Initial nr. of entries")
print(full_length)

subset_size = 10000

try:
    subset = int(sys.argv[1])
except:
    subset = 0
if subset*subset_size >= full_length:
    sys.exit('The subset is beyond the length of GALAH DR3')

galah_gaia_input = galah_gaia_input[subset*subset_size:np.min([(subset+1)*subset_size,full_length])]

# np.random.seed(123)
# shuffle = np.arange(len(galah_gaia_input))
# np.random.shuffle(shuffle)
# shuffle
# galah_gaia_input = (galah_gaia_input[shuffle])[:subset_size]

nr_galah_stars = len(galah_gaia_input['sobject_id'])
print("Nr. stars per subset")
print(nr_galah_stars)

nr_galah_stars_dynamics = np.where(
    np.isfinite(galah_gaia_input['ra']) &
    np.isfinite(galah_gaia_input['dec']) &
    np.isfinite(galah_gaia_input['r_est']) &
    np.isfinite(galah_gaia_input['pmra']) &
    np.isfinite(galah_gaia_input['pmdec']) &
    #np.isfinite(galah_gaia_input['rv_guess']) &
    np.isfinite(galah_gaia_input['rv_galah']) &
    np.isfinite(galah_gaia_input['ra_error']) &
    np.isfinite(galah_gaia_input['dec_error']) &
    np.isfinite(galah_gaia_input['r_hi']) &
    np.isfinite(galah_gaia_input['r_lo']) &
    np.isfinite(galah_gaia_input['pmra_error']) &
    np.isfinite(galah_gaia_input['pmdec_error']) &
    #np.isfinite(galah_gaia_input['e_rv_guess']) &
    #(galah_gaia_input['rv_guess'] != 999.) &
    #(galah_gaia_input['rv_guess'] != 1024.) &
    np.isfinite(galah_gaia_input['cov_e_rv_galah'])
    )[0]

# This should only me activated for tests with subsets of GALAH DR3
#nr_galah_stars_dynamics = nr_galah_stars_dynamics[:100]

galah_gaia = galah_gaia_input[nr_galah_stars_dynamics]
nr_stars = len(galah_gaia['sobject_id'])
print("Selected number of stars")
print(nr_stars)

In [ ]:
six_dimensions = {}

# Right ascension [deg]
six_dimensions['ra'] = galah_gaia['ra']
# Declination [deg]
six_dimensions['dec'] = galah_gaia['dec']

# Distance from Sun [pc]
six_dimensions['distance'] = galah_gaia['r_est']
# Parallax [mas]
six_dimensions['parallax'] = galah_gaia['parallax']

# Total proper motion in direction of right ascension [mas/yr]
six_dimensions['pmra'] = galah_gaia['pmra']
# Total proper motion in direction of declination [mas/yr]
six_dimensions['pmdec'] = galah_gaia['pmdec']
# Radial velocity [km/s]
six_dimensions['vrad'] = galah_gaia['rv_galah']

In [ ]:
e_six_dimensions = {}

# Error of right ascension [mas] to [deg]
e_six_dimensions['ra'] = galah_gaia['ra_error']/(1000.*3600.)
# Error of declination [mas] to [deg]
e_six_dimensions['dec'] = galah_gaia['dec_error']/(1000.*3600.)
# Error of distance from Sun [pc]
e_six_dimensions['distance_high'] = galah_gaia['r_hi']
e_six_dimensions['distance_low']  = galah_gaia['r_lo']
    # We are currently sampling a 2-sided Gaussian because Bailer-Jones are only giving 16th/50th/86th percentiles.
    # Any idea how to improve that because of missing posteriors from Bailer-Jones?
# Error of parallax [mas]
e_six_dimensions['parallax']  = galah_gaia['parallax_error']
# Error of total proper motion in direction of right ascension [mas/yr]
e_six_dimensions['pmra'] = galah_gaia['pmra_error']
# Error of total proper motion in direction of declination [mas/yr]
e_six_dimensions['pmdec'] = galah_gaia['pmdec_error']
# Error of radial velocity [km/s]
e_six_dimensions['vrad'] = galah_gaia['e_rv_galah']

## Monte Carlo sampling of Orbits

In [ ]:
MC_size = 10000

np.random.seed(123)

XYZ_labels       = ['X_XYZ','Y_XYZ','Z_XYZ']
UVW_labels       = ['U_LSR','V_LSR','W_LSR']

Rphiz_labels     = ['R_Rzphi','phi_Rzphi','z_Rzphi']
vRphiz_labels    = ['vR_Rzphi','vphi_Rzphi','vz_Rzphi']

action_labels    = ['J_R','L_Z','J_Z']
ext_orbit_labels = ['ecc', 'zmax', 'R_peri', 'R_ap', 'Energy']

orbit_labels = np.concatenate((
    XYZ_labels,
    UVW_labels,
    Rphiz_labels,
    vRphiz_labels,
    action_labels,
    ext_orbit_labels    
    ))
print(orbit_labels)

### Samples

In [ ]:
def sample_6d_uncertainty(
    six_dimensions,
    e_six_dimensions,
    MC_size=MC_size,
    no_correlation = False,
    extreme_covariance = False
    ):
    
    """
    This function samples the 6D space with the given uncertainties.
    3 Options are available:
    
    if MC_size==1: assume no uncertainties
    
    if no_correlation==True: Sample 6D parameters independently
    
    if no_correlation==False: Use Gaia DR2 covariance matrix to sample 5D
    and GALAH vrad for 6th D
    """

    np.random.seed(123)
    
    MC_sample_6D = {}
    
    # Option 1: We assume no errors and simply return the actual parameters
    if MC_size == 1:
        for each_key in six_dimensions.keys():
            if each_key == 'distance':
                MC_sample_6D[each_key] = np.array([[six_dimensions[each_key][x]] for x in range(nr_stars)])/1000.
            else:
                MC_sample_6D[each_key] = np.array([[six_dimensions[each_key][x]] for x in range(nr_stars)])
    
    else:
        if no_correlation == True:
            # Option2: We sample the errors including the covariance matrix
        
            # Sampling the distances from Bailer-Jones assuming 2 separate Gaussian distributions
            distance_sigma_lo  = np.array([np.abs(np.random.normal(loc = 0., scale = six_dimensions['distance'] -  e_six_dimensions['distance_low'])) for i in range(MC_size)])
            distance_sigma_hi  = np.array([np.abs(np.random.normal(loc = 0., scale = e_six_dimensions['distance_high'] - six_dimensions['distance'])) for i in range(MC_size)])
            select_lo_hi = np.array([(np.random.uniform(0, 1, size=nr_stars) < 0.5).astype(float) for x in range(MC_size)])

            MC_sample_6D['ra']       = np.array([np.random.normal(loc=six_dimensions['ra'], scale=e_six_dimensions['ra']) for i in range(MC_size)]).T
            MC_sample_6D['dec']      = np.array([np.random.normal(loc=six_dimensions['dec'], scale=e_six_dimensions['dec']) for i in range(MC_size)]).T
            MC_sample_6D['distance'] = (six_dimensions['distance'] + select_lo_hi*distance_sigma_hi - (1-select_lo_hi)*distance_sigma_lo).clip(min=0).T/1000.

            MC_sample_6D['pmra']     = np.array([np.random.normal(loc=six_dimensions['pmra'], scale=e_six_dimensions['pmra']) for i in range(MC_size)]).T
            MC_sample_6D['pmdec']    = np.array([np.random.normal(loc=six_dimensions['pmdec'], scale=e_six_dimensions['pmdec']) for i in range(MC_size)]).T
            MC_sample_6D['vrad']     = np.array([np.random.normal(loc=six_dimensions['vrad'], scale=e_six_dimensions['vrad']) for i in range(MC_size)]).T
        
        else:
            # Option3: We sample the errors including the covariance matrix
        
            # Mean vector and covariance matrix
            mu = np.array(
                [six_dimensions['ra'],
                 six_dimensions['dec'],
                 six_dimensions['parallax'],
                 six_dimensions['pmra'],
                 six_dimensions['pmdec'],
                 six_dimensions['vrad']
                ])

            s00 = (e_six_dimensions['ra'])**2
            s11 = (e_six_dimensions['dec'])**2
            s22 = e_six_dimensions['parallax']**2
            s33 = e_six_dimensions['pmra']**2
            s44 = e_six_dimensions['pmdec']**2
            s55 = e_six_dimensions['vrad']**2

            s01 = (e_six_dimensions['ra']) * e_six_dimensions['dec'] * galah_gaia['ra_dec_corr']
            s02 = (e_six_dimensions['ra']) * e_six_dimensions['parallax'] * galah_gaia['ra_parallax_corr']
            s03 = (e_six_dimensions['ra']) * e_six_dimensions['pmra'] * galah_gaia['ra_pmra_corr']
            s04 = (e_six_dimensions['ra']) * e_six_dimensions['pmdec'] * galah_gaia['ra_pmdec_corr']
            s05 = 0.*np.ones(np.shape(galah_gaia['sobject_id'])[0])

            s12 = (e_six_dimensions['dec']) * e_six_dimensions['parallax'] * galah_gaia['dec_parallax_corr']
            s13 = (e_six_dimensions['dec']) * e_six_dimensions['pmra'] * galah_gaia['dec_pmra_corr']
            s14 = (e_six_dimensions['dec']) * e_six_dimensions['pmdec'] * galah_gaia['dec_pmdec_corr']
            s15 = 0.*np.ones(np.shape(galah_gaia['sobject_id'])[0])

            s23 = e_six_dimensions['parallax'] * e_six_dimensions['pmra'] * galah_gaia['parallax_pmra_corr']
            s24 = e_six_dimensions['parallax'] * e_six_dimensions['pmdec'] * galah_gaia['parallax_pmdec_corr']
            s25 = 0.*np.ones(np.shape(galah_gaia['sobject_id'])[0])

            s34 = e_six_dimensions['pmra'] * e_six_dimensions['pmdec'] * galah_gaia['pmra_pmdec_corr']
            s35 = 0.*np.ones(np.shape(galah_gaia['sobject_id'])[0])

            s45 = 0.*np.ones(np.shape(galah_gaia['sobject_id'])[0])

            sigma = np.array([
                [
                [s00[x], s01[x], s02[x], s03[x], s04[x], s05[x]],
                [s01[x], s11[x], s12[x], s13[x], s14[x], s15[x]],
                [s02[x], s12[x], s22[x], s23[x], s24[x], s25[x]],
                [s03[x], s13[x], s23[x], s33[x], s34[x], s35[x]],
                [s04[x], s14[x], s24[x], s34[x], s44[x], s45[x]],
                [s05[x], s15[x], s25[x], s35[x], s45[x], s55[x]]
                ] for x in range(np.shape(galah_gaia['sobject_id'])[0])
            ])
            
            sample = np.array([np.random.multivariate_normal(mu[:,x], sigma[x], size= MC_size) for x in range(np.shape(mu)[1])])

            print('Created MC_sample_6D with (Nr. entries, Nr. Samples, Dimensions):')
            print(np.shape(sample))
            
            MC_sample_6D['ra']       = sample[:,:,0] #in deg   #*np.pi/180. # in rad
            MC_sample_6D['dec']      = sample[:,:,1] #in deg   #*np.pi/180. # in rad
            MC_sample_6D['distance'] = 1./sample[:,:,2] # in kpc
            MC_sample_6D['pmra']     = sample[:,:,3]
            MC_sample_6D['pmdec']    = sample[:,:,4]
            MC_sample_6D['vrad']     = sample[:,:,5]

    return MC_sample_6D

In [ ]:
def get_coordinates_astropy(nr_stars):
    
    for each_star in range(nr_stars):

        #_____coordinate transformation_____
        #input coordinates as quantities with units:
        ra     = MC_sample_6D['ra'][each_star]           *u.deg
        dec    = MC_sample_6D['dec'][each_star]          *u.deg
        dist   = MC_sample_6D['distance'][each_star]     *u.kpc
        pm_ra  = MC_sample_6D['pmra'][each_star]         *u.mas/u.year
        pm_dec = MC_sample_6D['pmdec'][each_star]        *u.mas/u.year
        v_los  = MC_sample_6D['vrad'][each_star]         *u.km/u.s

        #setup frame object:
        icrs = apycoords.ICRS(ra=ra, 
                          dec=dec, 
                          distance=dist, 
                          pm_ra_cosdec=pm_ra, 
                          pm_dec=pm_dec, 
                          radial_velocity=v_los)

        #setup representation frame:
        gc = apycoords.Galactocentric(galcen_distance=np.sqrt(X_gc_sun_kpc**2+Z_gc_sun_kpc**2)*u.kpc,
                                  galcen_v_sun=coord.CartesianDifferential([-vX_gc_sun_kms,vY_gc_sun_kms,vZ_gc_sun_kms]*u.km/u.s),
                                  z_sun=Z_gc_sun_kpc*u.kpc)
        galcen = icrs.transform_to(gc)

        #Galactocentric Cartesian coordinates:
        x_kpc = galcen.x.to(u.kpc).value
        y_kpc = galcen.y.to(u.kpc).value

        #Galactocentric Cartesian velocities:
        u_kms = galcen.v_x.to(u.km/u.s).value
        v_kms = galcen.v_y.to(u.km/u.s).value
        w_kms = galcen.v_z.to(u.km/u.s).value

        #change to Galactocentric cylindrical coordinates:
        galcen.set_representation_cls(coord.CylindricalRepresentation,s=coord.CylindricalDifferential)

        R_kpc   = galcen.rho.to(u.kpc).value
        phi_rad = galcen.phi.to(u.rad).value
        z_kpc   = galcen.z.to(u.kpc).value

        vR_kms = galcen.d_rho.to(u.km/u.s).value
        vT_kms = -(galcen.d_phi.to(u.rad/u.s)*galcen.rho.to(u.km)  / (1.*u.radian)).value
        vz_kms = galcen.d_z.to(u.km/u.s).value

        interim_orbit_information[XYZ_labels[0]].append(x_kpc)
        interim_orbit_information[XYZ_labels[1]].append(y_kpc)
        interim_orbit_information[XYZ_labels[2]].append(z_kpc)

        interim_orbit_information[Rphiz_labels[0]].append(R_kpc)
        interim_orbit_information[Rphiz_labels[1]].append(phi_rad)
        interim_orbit_information[Rphiz_labels[2]].append(z_kpc)

        interim_orbit_information[UVW_labels[0]].append(u_kms)
        interim_orbit_information[UVW_labels[1]].append(v_kms)
        interim_orbit_information[UVW_labels[2]].append(w_kms)

        interim_orbit_information[vRphiz_labels[0]].append(vR_kms)
        interim_orbit_information[vRphiz_labels[1]].append(vT_kms)
        interim_orbit_information[vRphiz_labels[2]].append(vz_kms)

        jR,lz,jz = aAS(
            #R,vR,vT,z,vz[,phi]
            R_kpc/_REFR0, vR_kms/_REFV0, vT_kms/_REFV0, z_kpc/_REFV0, vz_kms/_REFV0, phi_rad,
        )

        interim_orbit_information[action_labels[0]].append(jR*_REFR0*_REFV0)
        interim_orbit_information[action_labels[1]].append(lz*_REFR0*_REFV0)
        interim_orbit_information[action_labels[2]].append(jz*_REFR0*_REFV0)

        ezr1, ezr2, ezr3, ezr4 = aAS.EccZmaxRperiRap(
            #R,vR,vT,z,vz[,phi]
            R_kpc/_REFR0, vR_kms/_REFV0, vT_kms/_REFV0, z_kpc/_REFV0, vz_kms/_REFV0, phi_rad
        )
        
        o= Orbit(vxvv=[R_kpc/_REFR0, vR_kms/_REFV0, vT_kms/_REFV0, z_kpc/_REFV0, vz_kms/_REFV0, phi_rad],ro=_REFR0,vo=_REFV0)
        energy = o.E(pot=pot)

        interim_orbit_information[ext_orbit_labels[0]].append(ezr1)
        interim_orbit_information[ext_orbit_labels[1]].append(ezr2*_REFR0)
        interim_orbit_information[ext_orbit_labels[2]].append(ezr3*_REFR0)
        interim_orbit_information[ext_orbit_labels[3]].append(ezr4*_REFR0)
        interim_orbit_information[ext_orbit_labels[4]].append(energy)

In [ ]:
def get_coordniates_galpy(nr_stars):
    for each_star in range(nr_stars):

        lb = bovy_coords.radec_to_lb(
                    MC_sample_6D['ra'][each_star],
                    MC_sample_6D['dec'][each_star],
                    degree=True,
                    epoch=None
                    )
        l_deg = lb[:,0]
        b_deg = lb[:,1]
        
        #_____(pm_ra,pm_dec) --> (pm_l,pm_b)_____
        pmlpmb = bovy_coords.pmrapmdec_to_pmllpmbb(
                            MC_sample_6D['pmra'][each_star],
                            MC_sample_6D['pmdec'][each_star],
                            MC_sample_6D['ra'][each_star],
                            MC_sample_6D['dec'][each_star],
                            degree=True,
                            epoch=None
                            )
        pml_masyr = pmlpmb[:,0]
        pmb_masyr = pmlpmb[:,1]

        #_____(l,b,d) --> Galactic, heliocentric cartesian coordinates (X,Y,Z)_____
        xyz = bovy_coords.lbd_to_XYZ(
                        l_deg,b_deg,
                        MC_sample_6D['distance'][each_star],
                        degree=True)
        X_HC_kpc = xyz[:,0]
        Y_HC_kpc = xyz[:,1]
        Z_HC_kpc = xyz[:,2]

        #_____(v_los,pm_l,pm_b) & (l,b,d) --> (vx,vy,vz)______
        vxvyvz = bovy_coords.vrpmllpmbb_to_vxvyvz(
                        MC_sample_6D['vrad'][each_star],
                        pml_masyr,pmb_masyr,
                        l_deg,b_deg,
                        MC_sample_6D['distance'][each_star],
                        XYZ=False,degree=True
                        )
        U_HC_kms = vxvyvz[:,0]
        V_HC_kms = vxvyvz[:,1]
        W_HC_kms = vxvyvz[:,2]
        
        #______(X,Y,Z) --> Galactic, Galactocentric cylindrical coordinates (R,phi,z)______:
        Rzphi= bovy_coords.XYZ_to_galcencyl(
                        X_HC_kpc, Y_HC_kpc, Z_HC_kpc, 
                        Xsun=X_gc_sun_kpc,Zsun=Z_gc_sun_kpc
                        )
        R_kpc   = Rzphi[:,0]
        phi_rad = Rzphi[:,1]
        z_kpc   = Rzphi[:,2]

        #______(vx,vy,vz) & (x,y,z) --> (vR,vT,vz)______
        vRvTvZ = bovy_coords.vxvyvz_to_galcencyl(
                        U_HC_kms, 
                        V_HC_kms, 
                        W_HC_kms, 
                        R_kpc,
                        phi_rad, 
                        z_kpc,
                        Xsun=X_gc_sun_kpc,Zsun=Z_gc_sun_kpc,
                        vsun=[vX_gc_sun_kms,vY_gc_sun_kms,vZ_gc_sun_kms], 
                        galcen=True
                        )
        vR_kms = vRvTvZ[:,0]
        vT_kms = vRvTvZ[:,1]
        vz_kms = vRvTvZ[:,2]
        
        jR,lz,jz = aAS(
            #R,vR,vT,z,vz[,phi]
            R_kpc/_REFR0, vR_kms/_REFV0, vT_kms/_REFV0, z_kpc/_REFV0, vz_kms/_REFV0, phi_rad,
        )

        ezr1, ezr2, ezr3, ezr4 = aAS.EccZmaxRperiRap(
            #R,vR,vT,z,vz[,phi]
            R_kpc/_REFR0, vR_kms/_REFV0, vT_kms/_REFV0, z_kpc/_REFV0, vz_kms/_REFV0, phi_rad
        )
        
        o= Orbit(vxvv=np.array([R_kpc/_REFR0, vR_kms/_REFV0, vT_kms/_REFV0, z_kpc/_REFV0, vz_kms/_REFV0, phi_rad]).T,ro=_REFR0,vo=_REFV0)
        energy = o.E(pot=pot)

        interim_orbit_information[XYZ_labels[0]].append(X_HC_kpc)
        interim_orbit_information[XYZ_labels[1]].append(Y_HC_kpc)
        interim_orbit_information[XYZ_labels[2]].append(Z_HC_kpc)

        interim_orbit_information[UVW_labels[0]].append(U_HC_kms)
        interim_orbit_information[UVW_labels[1]].append(V_HC_kms)
        interim_orbit_information[UVW_labels[2]].append(W_HC_kms)

        interim_orbit_information[Rphiz_labels[0]].append(R_kpc)
        interim_orbit_information[Rphiz_labels[1]].append(phi_rad)
        interim_orbit_information[Rphiz_labels[2]].append(z_kpc)

        interim_orbit_information[vRphiz_labels[0]].append(vR_kms)
        interim_orbit_information[vRphiz_labels[1]].append(vT_kms)
        interim_orbit_information[vRphiz_labels[2]].append(vz_kms)
        
        interim_orbit_information[action_labels[0]].append(jR*_REFR0*_REFV0)
        interim_orbit_information[action_labels[1]].append(lz*_REFR0*_REFV0)
        interim_orbit_information[action_labels[2]].append(jz*_REFR0*_REFV0)

        interim_orbit_information[ext_orbit_labels[0]].append(ezr1)
        interim_orbit_information[ext_orbit_labels[1]].append(ezr2*_REFR0)
        interim_orbit_information[ext_orbit_labels[2]].append(ezr3*_REFR0)
        interim_orbit_information[ext_orbit_labels[3]].append(ezr4*_REFR0)
        interim_orbit_information[ext_orbit_labels[4]].append(energy)

# Compute orbit information

In [ ]:
# The final orbit information will go into the following dictionary:

orbit_information = collections.OrderedDict()

# Compute best orbit information

In [ ]:
interim_orbit_information = collections.OrderedDict()
for each_orbit_label in orbit_labels:
    interim_orbit_information[each_orbit_label] = []
    #interim_orbit_information[each_orbit_label+'_sig'] = []

# We will first sample without uncertainty
MC_sample_6D = sample_6d_uncertainty(six_dimensions,e_six_dimensions,MC_size=1,no_correlation=True)

#get_coordinates_astropy(nr_stars)
get_coordniates_galpy(nr_stars)

for each_orbit_label in orbit_labels:
    interim_orbit_information[each_orbit_label] = np.array(interim_orbit_information[each_orbit_label])
    try:
        orbit_information[each_orbit_label] = np.array([np.nanmean(interim_orbit_information[each_orbit_label][each_star,:]) for each_star in range(nr_stars)])
    except:
        print('no output for '+each_orbit_label)

# Compute uncertainties via MC sampling

In [ ]:
interim_orbit_information = collections.OrderedDict()
for each_orbit_label in orbit_labels:
    interim_orbit_information[each_orbit_label] = []
    interim_orbit_information[each_orbit_label+'_5'] = []
    interim_orbit_information[each_orbit_label+'_50'] = []
    interim_orbit_information[each_orbit_label+'_95'] = []

# And now we sample with the Gaia DR2 covariance matrices
MC_sample_6D = sample_6d_uncertainty(six_dimensions,e_six_dimensions,MC_size=MC_size,no_correlation=False)

#get_coordinates_astropy(nr_stars)
get_coordniates_galpy(nr_stars)
        
for each_orbit_label in orbit_labels:
    interim_orbit_information[each_orbit_label] = np.array(interim_orbit_information[each_orbit_label])
    try:
        percentiles = np.array([np.percentile(interim_orbit_information[each_orbit_label][each_star,:], q=[5,50,95]) for each_star in range(nr_stars)])
        #orbit_information[each_orbit_label+'_mu'] = np.array([np.nanmean(interim_orbit_information[each_orbit_label][each_star,:]) for each_star in range(nr_stars)])
        #orbit_information[each_orbit_label+'_sig'] = np.array([np.nanstd(interim_orbit_information[each_orbit_label][each_star,:]) for each_star in range(nr_stars)])
        orbit_information[each_orbit_label+'_5'] = percentiles[:,0]
        orbit_information[each_orbit_label+'_50'] = percentiles[:,1]
        orbit_information[each_orbit_label+'_95'] = percentiles[:,2]
    except:
        print('no output for '+each_orbit_label)

In [ ]:
def plot_sampling(data, star_index=1):
    f = plt.figure(figsize=(15,10))

    hist_kwarfs=dict(bins=25,cmin=1)
    hist_k = dict(bins=25)
    
    ax=plt.subplot(6,6,31)
    ax.hist(data['ra'][star_index],**hist_k);
    ax.set_xlabel('ra')
    ax.set_ylabel('ra')
    ax.set_xticks([])
    ax.set_yticks([])

    ax=plt.subplot(6,6,32)
    ax.hist2d(data['dec'][star_index],data['ra'][star_index],**hist_kwarfs);
    ax.set_xlabel('dec')
    ax.set_xticks([])
    ax=plt.subplot(6,6,33)
    ax.hist2d(data['distance'][star_index],data['ra'][star_index],**hist_kwarfs);
    ax.set_xlabel('distance')
    ax.set_yticks([])
    ax=plt.subplot(6,6,34)
    ax.hist2d(data['pmra'][star_index],data['ra'][star_index],**hist_kwarfs);
    ax.set_xlabel('pmra')
    ax.set_yticks([])
    ax=plt.subplot(6,6,35)
    ax.hist2d(data['pmdec'][star_index],data['ra'][star_index],**hist_kwarfs);
    ax.set_xlabel('pmdec')
    ax.set_yticks([])
    ax=plt.subplot(6,6,36)
    ax.hist2d(data['vrad'][star_index],data['ra'][star_index],**hist_kwarfs);
    ax.set_xlabel('rv')
    ax.set_yticks([])

    ax=plt.subplot(6,6,26)
    ax.set_ylabel('dec')
    ax.hist(data['dec'][star_index],**hist_k);
    ax.set_yticks([])
    ax=plt.subplot(6,6,27)
    ax.hist2d(data['distance'][star_index],data['dec'][star_index],**hist_kwarfs);
    ax.set_yticks([])
    ax=plt.subplot(6,6,28)
    ax.hist2d(data['pmra'][star_index],data['dec'][star_index],**hist_kwarfs);
    ax.set_yticks([])
    ax=plt.subplot(6,6,29)
    ax.hist2d(data['pmdec'][star_index],data['dec'][star_index],**hist_kwarfs);
    ax.set_yticks([])
    ax=plt.subplot(6,6,30)
    ax.hist2d(data['vrad'][star_index],data['dec'][star_index],**hist_kwarfs);
    ax.set_yticks([])

    ax=plt.subplot(6,6,21)
    ax.set_ylabel('parallax')
    ax.hist(data['distance'][star_index],**hist_k);
    ax=plt.subplot(6,6,22)
    ax.hist2d(data['pmra'][star_index],data['distance'][star_index],**hist_kwarfs);
    ax=plt.subplot(6,6,23)
    ax.hist2d(data['pmdec'][star_index],data['distance'][star_index],**hist_kwarfs);
    ax=plt.subplot(6,6,24)
    ax.hist2d(data['vrad'][star_index],data['distance'][star_index],**hist_kwarfs);

    ax=plt.subplot(6,6,16)
    ax.set_ylabel('pmra')
    ax.hist(data['pmra'][star_index],**hist_k);
    ax=plt.subplot(6,6,17)
    ax.hist2d(data['pmdec'][star_index],data['pmra'][star_index],**hist_kwarfs);
    ax=plt.subplot(6,6,18)
    ax.hist2d(data['vrad'][star_index],data['pmra'][star_index],**hist_kwarfs);

    ax=plt.subplot(6,6,11)
    ax.hist(data['pmdec'][star_index],**hist_k);
    ax.set_ylabel('pmdec')
    ax=plt.subplot(6,6,12)
    ax.hist2d(data['vrad'][star_index],data['pmdec'][star_index],**hist_kwarfs);

    ax=plt.subplot(6,6,6)
    ax.set_ylabel('vrad')
    ax.hist(data['vrad'][star_index],**hist_k);

    plt.tight_layout()

if debug==True:
    plot_sampling(MC_sample_6D, star_index = 0)

In [ ]:
if debug==True:
    
    star_index = 2
    
    print("XYZ = ({x:8.2f},{y:8.2f},{z:8.2f}) [kpc]".format(
        x=orbit_information[XYZ_labels[0]][star_index],
        y=orbit_information[XYZ_labels[1]][star_index],
        z=orbit_information[XYZ_labels[2]][star_index]
        ))
          
    print("UVW = ({u:8.2f},{v:8.2f},{w:8.2f}) [kpc km/s]".format(
        u=orbit_information[UVW_labels[0]][star_index],
        v=orbit_information[UVW_labels[1]][star_index],
        w=orbit_information[UVW_labels[2]][star_index]
        ))

    print(r"R   = {r:6.2f} -{r_minus:6.2f} + {r_plus:6.2f} [kpc]".format(
            r=orbit_information[Rphiz_labels[0]][star_index],
            r_minus=orbit_information[Rphiz_labels[0]][star_index] - orbit_information[Rphiz_labels[0]+'_5'][star_index],
            r_plus=orbit_information[Rphiz_labels[0]+'_95'][star_index] - orbit_information[Rphiz_labels[0]][star_index]
            ))
    print(r"phi = {phi:6.2f} -{phi_minus:6.2f} + {phi_plus:6.2f} [kpc]".format(
            phi=orbit_information[Rphiz_labels[1]][star_index],
            phi_minus=orbit_information[Rphiz_labels[1]][star_index] - orbit_information[Rphiz_labels[1]+'_5'][star_index],
            phi_plus=orbit_information[Rphiz_labels[1]+'_95'][star_index] - orbit_information[Rphiz_labels[1]][star_index]
            ))
    print(r"z = {z:6.2f} -{z_minus:6.2f} + {z_plus:6.2f} [kpc]".format(
            z=orbit_information[Rphiz_labels[2]][star_index],
            z_minus=orbit_information[Rphiz_labels[2]][star_index] - orbit_information[Rphiz_labels[2]+'_5'][star_index],
            z_plus=orbit_information[Rphiz_labels[2]+'_95'][star_index] - orbit_information[Rphiz_labels[2]][star_index]
            ))

    print("J_R = {jr:6.2f} - {jr_m:6.2f} + {jr_p:6.2f} [kpc km/s]".format(
        jr=orbit_information['J_R'][star_index],
        jr_m=orbit_information['J_R'][star_index]-orbit_information['J_R_5'][star_index],
        jr_p=orbit_information['J_R_95'][star_index]-orbit_information['J_R'][star_index]
        ))
    print("L_Z = {lz:6.2f} - {lz_m:6.2f} + {lz_p:6.2f} [kpc km/s]".format(
        lz=orbit_information['L_Z'][star_index],
        lz_m=orbit_information['L_Z'][star_index]-orbit_information['L_Z_5'][star_index],
        lz_p=orbit_information['L_Z_95'][star_index]-orbit_information['L_Z'][star_index]
        ))
    print("J_Z = {jz:6.2f} - {jz_m:6.2f} + {jz_p:6.2f} [kpc km/s]".format(
        jz=orbit_information['J_Z'][star_index],
        jz_m=orbit_information['J_Z'][star_index]-orbit_information['J_Z_5'][star_index],
        jz_p=orbit_information['J_Z_95'][star_index]-orbit_information['J_Z'][star_index]
        ))
    print("e = {ecc:6.2f}, zmax = {zmax:6.2f}, Rperi = {rperi:6.2f}, Rapo = {rapo:6.2f}".format(
        ecc=orbit_information['ecc'][star_index],
        zmax=orbit_information['zmax'][star_index],
        rperi=orbit_information['R_peri'][star_index],
        rapo=orbit_information['R_ap'][star_index],
        ))

In [ ]:
if debug==True: 
    
    useful = np.isfinite(galah_gaia['parallax'])
    #useful = (galah_gaia['parallax_error']/galah_gaia['parallax'] < 0.3) & (galah_gaia['parallax'] > 0)
    
    errorbar_kwargs = dict(fmt='o', rasterized=True, ms=1, c='r')
    
    f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2,3,figsize=(15,10))

    interim_orbit_information['Toomre'] = np.sqrt(interim_orbit_information['U_LSR']**2 + interim_orbit_information['W_LSR']**2)
    toomre_p = np.percentile(interim_orbit_information['Toomre'], q=[5,50,95], axis=1)
    
    orbit_information['Toomre'] = np.sqrt(orbit_information['U_LSR']**2 + orbit_information['W_LSR']**2)
    orbit_information['Toomre_5'] = toomre_p[0]
    orbit_information['Toomre_50'] = toomre_p[1]
    orbit_information['Toomre_95'] = toomre_p[2]
    
    def plot_distribution(xlabel, ylabel, yscale, ax):
        ax=ax
        if yscale == 'lin':
            ax.scatter(
                interim_orbit_information[xlabel][useful],
                interim_orbit_information[ylabel][useful],
                s=0.5,alpha=0.01,rasterized=True,
                label='MC'
            )
            ax.errorbar(
            orbit_information[xlabel+'_50'][useful],
            orbit_information[ylabel+'_50'][useful],
            xerr=[orbit_information[xlabel+'_50'][useful]-orbit_information[xlabel+'_5'][useful],orbit_information[xlabel+'_95'][useful]-orbit_information[xlabel+'_50'][useful]],
            yerr=[orbit_information[ylabel+'_50'][useful]-orbit_information[ylabel+'_5'][useful],orbit_information[ylabel+'_95'][useful]-orbit_information[ylabel+'_50'][useful]],
            label='5/50/95',
            **errorbar_kwargs
            )
            ax.scatter(
            orbit_information[xlabel][useful],
            orbit_information[ylabel][useful],
            c='k',rasterized=True,
            zorder=2,
            label='Best'
            )
        elif yscale == 'sqrt':
            ax.scatter(
                interim_orbit_information[xlabel][useful],
                np.sqrt(interim_orbit_information[ylabel][useful]),
                s=0.5,alpha=0.01,rasterized=True
            )
            ax.errorbar(
            orbit_information[xlabel+'_50'][useful],
            np.sqrt(orbit_information[ylabel+'_50'][useful]),
            xerr=[
                    orbit_information[xlabel+'_50'][useful]-orbit_information[xlabel+'_5'][useful],
                    orbit_information[xlabel+'_95'][useful]-orbit_information[xlabel+'_50'][useful]],
            yerr=[
                    np.sqrt(orbit_information[ylabel+'_50'][useful])-np.sqrt(orbit_information[ylabel+'_5'][useful]),
                    np.sqrt(orbit_information[ylabel+'_95'][useful])-np.sqrt(orbit_information[ylabel+'_50'][useful])
                ],
            **errorbar_kwargs
            )
            ax.scatter(
            orbit_information[xlabel][useful],
            np.sqrt(orbit_information[ylabel][useful]),
            c='k',rasterized=True,
            zorder=2
            )

    plot_distribution(xlabel=XYZ_labels[0], ylabel=XYZ_labels[1], yscale='lin', ax=ax1)
    plot_distribution(xlabel=Rphiz_labels[0], ylabel=Rphiz_labels[2], yscale='lin', ax=ax2)
    plot_distribution(xlabel=UVW_labels[0], ylabel='Toomre', yscale='lin', ax=ax3)
    plot_distribution(xlabel=ext_orbit_labels[0], ylabel=ext_orbit_labels[1], yscale='lin', ax=ax4)
    plot_distribution(xlabel=ext_orbit_labels[2], ylabel=ext_orbit_labels[3], yscale='lin', ax=ax5)
    plot_distribution(xlabel=action_labels[1], ylabel=action_labels[0], yscale='sqrt', ax=ax6)

    ax1.set_xlabel('X (XYZ) [kpc]')
    ax1.set_ylabel('Y (XYZ) [kpc]')

    ax2.set_xlabel('R (GC) [kpc]')
    ax2.set_ylabel('z (GC) [kpc]')
    legend = ax2.legend(loc='upper right',fontsize=15, markerscale=2)
    for ind, legend_handle in enumerate(legend.legendHandles):
        if ind==0:
            legend_handle.set_alpha(1)

    ax3.set_xlim(-700,200)
    ax3.set_ylim(0,450)
    ax3.set_xlabel('Toomre V (LSR) [km/s]')
    ax3.set_ylabel('Toomre UW (LSR) [km/s]')

    ax4.set_xlabel('Eccentricity')
    ax4.set_ylabel(r'$z_\text{max}$ [kpc]')

    ax5.set_xlabel(r'R (pericenter) [kpc]')
    ax5.set_ylabel(r'R (apocenter) [kpc]')

    ax6.set_xlabel(r'$L_Z$ [kpc km/s]')
    ax6.set_ylabel(r'$\sqrt{J_R \mathrm{[kpc km/s]}}$')
    ax6.set_ylim(-10,75)
    ax6.set_xlim(-1000,3000)

    plt.tight_layout()
    
    plt.savefig('figures/MC_output.pdf',dpi=300,bbox_inches='tight')

In [ ]:
galah_dynamics = collections.OrderedDict()

galah_dynamics['sobject_id'] = galah_gaia_input['sobject_id']
for each_orbit_label in orbit_labels:
    galah_dynamics[each_orbit_label] = np.zeros(nr_galah_stars, dtype=float)
    galah_dynamics[each_orbit_label].fill(np.nan)
    (galah_dynamics[each_orbit_label])[nr_galah_stars_dynamics] = orbit_information[each_orbit_label]

for each_orbit_label in orbit_labels:
    for each_sampler in ['5','50','95']:
        galah_dynamics[each_orbit_label+'_'+each_sampler] = np.zeros(nr_galah_stars, dtype=float)
        galah_dynamics[each_orbit_label+'_'+each_sampler].fill(np.nan)
        (galah_dynamics[each_orbit_label+'_'+each_sampler])[nr_galah_stars_dynamics] = orbit_information[each_orbit_label+'_'+each_sampler]

galah_dynamics_data = pandas.DataFrame(galah_dynamics,columns=galah_dynamics.keys())
galah_dynamics_data

data_for_fits = Table.from_pandas(galah_dynamics_data)
data_for_fits.write('dynamics_output/sobject_dynamics_'+str(subset)+'.fits',overwrite=True)

In [ ]:
try:
    import email
    import email.mime.application
    from email.MIMEMultipart import MIMEMultipart
    from email.MIMEText import MIMEText
    from email.MIMEImage import MIMEImage
    msg = MIMEMultipart()

    msg['From'] = 'gemini2'
    msg['To'] = 'buder@mpia.de'
    msg['Subject'] = 'Orbit calculation for subset '+str(subset)+' GALAH DR3 finished'                                                                                                         

    import smtplib
    mailer = smtplib.SMTP('localhost')
    mailer.sendmail('gemini2', 'buder@mpia.de', msg.as_string())
    mailer.close()
    print('Email sent')
except:
    print('Could not send email')